# Codebase for "A Dynamic Theory of Deterrence and Compliance"

Format: JupyterLab Notebook

Kernel: Python3

Code for all results and figures. 

*Note, to use this file user must first set the local path to store results of computation.*

In [1]:
""" 
    import libraries and set display options 
"""
import sys
import matplotlib.collections

import random as rn
import numpy as np
import math as math
import quantecon as qe
import scipy.stats as stats
import statsmodels.api as sm
import matplotlib.pyplot as plt

from numba import jit
from scipy.stats import norm
from scipy.stats import binom
from scipy.stats import truncnorm
from scipy.stats import entropy
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition

path_data = 
path_code = 
path_figs = 

# aesthetic settings
import warnings
sys.setswitchinterval(1500)
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
""" 
    baseline model parameters for kernel
    
"""
# main parameters
agents = 100
Z = 1, 2

# distribution parameters
ḡ , σ = 0.6 , 0.2

# Bayesian priors
α, β = 1, 0.25

# costs
ρ, λ = 2 , 5

# domain of analysis for policy
F = 1
R_low, R_high = 0, agents+1

# benchmarking parameters
block = 50000
checks, C = 5, 0.01

In [3]:
""" 
    functions used by simulations 
"""

# apprehension function, min catch
@jit(nopython=True)
def Apprehend(v, R):
    γ = 0.80 
    if v == 0:
        prob_a = γ
    else:
        prob_a = γ*min(1, R/v)
    return prob_a

# apprehension function, exponential
@jit(nopython=True)
def Apprehend_exp(v, R):
    γ = 0.80  
    ϵ = 8
    if v == 0:
        prob_a = γ
    else:
        prob_a = γ*(1-(1/pow(ϵ,(R/v))))
    return prob_a

# create a 'criminal opportunity'
def Opportunity():
    lower, upper = 0, 1
    ḡ , σ = 0.6, 0.2
    return max(0, np.random.normal(ḡ, σ)) 

# create a distribution of 'criminal opportunities'
@jit(nopython=True)
def Criminal_opps(μ, σ, agents, T):
    g = np.zeros((T,agents))
    for i in range(T):
        g[i] = [max(0, np.random.normal(ḡ, σ)) for agent in range(agents)]
    return g

# create a distribution of 'criminal opportunities'
@jit(nopython=True)
def Criminal_opps_uniform(low, high, agents, T):
    g = np.zeros((T,agents))
    for i in range(T):
        g[i] = [np.random.uniform(low, high) for agent in range(agents)]
    return g

def Simulation(criminal_opps, R, F, Z):
# agent priors
    α,β = 1, 0.25
    init_period = 100
# tracking arrays for endogenous variables of interest
    a_t,v_t, g_t, q_t = [], [], [], []
# seed for priors
    append_v = v_t.append
    append_a = a_t.append
    append_g = g_t.append
    append_q = q_t.append
    for t in range(Z):
        v = np.random.randint(1,agents)
        a = np.random.randint(0,v)
        append_v(v)
        append_a(a)
# main sim loop, starts at T = 500   
    for t in range(len(criminal_opps)):
        a, v, g, q = 0, 0, 0, 0
        q = (α + np.sum(a_t[-Z:]))/(α + β + np.sum(v_t[-Z:]))
        for g_i in criminal_opps[t]:
            if  q <= g_i/F:
                v = v + 1
                g = g + g_i
        a = stats.binom.rvs(v, Apprehend(v, R))
        append_v(v)
        append_a(a)
        append_g(g)
        append_q(q)
    
    return np.array(v_t[init_period+Z:]), np.array(a_t[init_period+Z:]), np.array(g_t[init_period+Z:]), np.array(q_t[init_period+Z:])

def Simulation_exp(criminal_opps, R, F, Z):
# agent priors
    α,β = 1, 0.25
    init_period = 100
# tracking arrays for endogenous variables of interest
    a_t,v_t, g_t, q_t = [], [], [], []
# seed for priors
    append_v = v_t.append
    append_a = a_t.append
    append_g = g_t.append
    append_q = q_t.append
    for t in range(Z):
        v = np.random.randint(1,agents)
        a = np.random.randint(0,v)
        append_v(v)
        append_a(a)
# main sim loop, starts at T = 500   
    for t in range(len(criminal_opps)):
        a,v,g = 0,0,0
        q = (α + np.sum(a_t[-Z:]))/(α + β + np.sum(v_t[-Z:]))
        for g_i in criminal_opps[t]:
            if  q <= g_i/F:
                v = v + 1
                g = g + g_i
        a = stats.binom.rvs(v, Apprehend_exp(v, R))
        append_v(v)
        append_a(a)
        append_g(g)
        append_q(q)
    
    return np.array(v_t[init_period+Z:]), np.array(a_t[init_period+Z:]), np.array(g_t[init_period+Z:]), np.array(q_t[init_period+Z:])

def Simulation_het_q(criminal_opps, R, F, Z):
# agent priors
    α,β = 1, 0.25
    init_period = 100
# tracking arrays for endogenous variables of interest
    a_t,v_t, g_t, q_t = [], [], [], []
# seed for priors
    append_v = v_t.append
    append_a = a_t.append
    append_g = g_t.append
    append_q = q_t.append
    for t in range(Z):
        v = np.random.randint(1,agents)
        a = np.random.randint(0,v)
        append_v(v)
        append_a(a)
# main sim loop, starts at T = 500   
    for t in range(len(criminal_opps)):
        a,v,g = 0,0,0
        q = (α + np.sum(a_t[-Z:]))/(α + β + np.sum(v_t[-Z:]))
        for g_i in criminal_opps[t]:
            q_i = q + np.random.uniform(-0.2,0.2)
            if q_i > 1:
                q_i = 1
            if q_i < 0:
                q_i = 0
            if  q_i <= g_i/F:
                v = v + 1
                g = g + g_i
        a = stats.binom.rvs(v, Apprehend(v, R))
        append_v(v)
        append_a(a)
        append_g(g)
        append_q(q)
    
    return np.array(v_t[init_period+Z:]), np.array(a_t[init_period+Z:]), np.array(g_t[init_period+Z:]), np.array(q_t[init_period+Z:])

def Crackdown(criminal_opps, BE, RGB, RBB, F, Z):
# agent priors
    T = len(criminal_opps)
    α,β = 1, 0.25
    r = RGB
# tracking arrays for endogenous variables of interest
    a_t,v_t, g_t, r_t = [], [], [],[]
# seed for priors
    append_v = v_t.append
    append_a = a_t.append
    append_g = g_t.append
    append_r = r_t.append
    for t in range(Z):
        v = np.random.randint(1,agents)
        a = stats.binom.rvs(v, Apprehend(v, r))
        append_v(v)
        append_a(a)
        append_r(r)
# main sim loop, starts at T = 500   
    for t in range(len(criminal_opps)):
        if t > 0:
            if v >= BE:
                r = RBB
            else:
                r = RGB 
        else:
            r = RGB
        a,v,g = 0,0,0
        q = (α + np.sum(a_t[-Z:]))/(α + β + np.sum(v_t[-Z:]))
        for g_i in criminal_opps[t]:
            if  q <= g_i/F:
                v = v + 1
                g = g + g_i
        a = stats.binom.rvs(v, Apprehend(v, r))
        append_v(v)
        append_a(a)
        append_g(g)
        append_r(r)
    
    return np.array(v_t[-T:]), np.array(a_t[-T:]), np.array(g_t[-T:]), np.array(r_t[-T:])

# State Space Production and Transition Matrices

The first function produces, for a given number of agents, the State Space of the model ($\boldsymbol{Z}$) from Section 2.3.

The second function produces the conditional probabilities establishing Proposition 3 on page 11.

The third function takes the probabilities from above as arguments and produces the complete transition matrix for the model ($\boldsymbol{T}$ from page 9).

In [4]:
"""
    Produce State space
        parameters: agents
        returns: S, Shat, s
"""
def State_space(agents: int):
    w = np.sum(np.arange(1,agents+2))
    s = np.zeros(w, dtype=int)
    R̂ = np.empty(w, dtype=object)
    Ŝ = np.empty(w, dtype=object)
    S = [[(i,j) for j in range(agents+1)] for i in range(agents+1)]
    l=0

    for i in range(agents+1):
        for j in range(agents+1):
            if i >= j:
                k = np.arange(i+1)
                S[i][j] = (i,j)
                Ŝ[l] = (1+i-j+np.sum(k),i,j)
                s[l] = 1+i-j+np.sum(k)
                l=l+1
            if i < j:
                S[i][j] = (i,0)
    Ŝ = sorted(Ŝ, key=lambda tup: tup[0])
    s = sorted(s)
    return S, Ŝ, s

"""
    Produce M1 and M2 matrices to speed analysis
        parameters: agents, F
        returns: M1, M2
"""

def TM_matrices(agents: int, F:float):
# parameters
    ḡ = 0.60
    σ = 0.20
    α,β = 1,0.25
# tracking arrays/vectors
    S, Ŝ, s = State_space(agents)
    w = len(s)
    A = np.zeros(w)
    π = np.zeros(w)
    M1 = np.zeros((w,w))
    M2 = np.zeros((w,w))
    
# main loop
    for state_1 in range(w):
        v = Ŝ[state_1][1]
        a = Ŝ[state_1][2]
        q = (α + a)/(α + β + v)
        π[state_1] = 1 - norm.cdf(F*q-ḡ,0,σ)
        for state_2 in range(w):
            v = Ŝ[state_2][1]
            a = Ŝ[state_2][2]
            v̂ = math.factorial(v)
            â = math.factorial(a)
            d̂ = math.factorial(v-a)
            M1[state_1,state_2] = binom.pmf(v,agents,π[state_1])
            M2[state_1,state_2] = v̂/(â*d̂)
    return M1, M2

""" 
    Produce transition matrix for all s to s'
        s = numbered states
"""
def Transition_matrix(agents: int, M1, M2, R: int, F:float, switch:bool):
# tracking arrays/vectors
    S, Ŝ, s = State_space(agents)
    w = len(s)
    T̂ = np.zeros((w,w))
    
# main loop
    for state_1 in range(w):
        for state_2 in range(w):
            v = Ŝ[state_2][1]  
            a = Ŝ[state_2][2]
            d = v - a
            if switch:
                Â = Apprehend(v, R[state_1])
            else:
                Â = Apprehend(v, R[state_1])
            M3 = math.pow(Â, a)*math.pow(1-Â, d)
            T̂[state_1,state_2] = M1[state_1,state_2]*M2[state_1,state_2]*M3
    return T̂

def Compute_cost(agents: int, R:int , F:float):
# parameters
    δ, λ = 2, 5
    ḡ = 0.60
    σ = 0.20
    α,β = 1,0.25
# tracking arrays/vectors
    S, Ŝ, s = State_space(agents)
    w = len(s)
    A = np.zeros(w)
    π = np.zeros(w)
    Ĉ = np.zeros(w)
    
# main loop
    for state in range(w):
        v = Ŝ[state][1]
        a = Ŝ[state][2]
        q = (α + a)/(α + β + v)
        π[state] = 1 - norm.cdf(F*q-ḡ, 0, σ)
        A[state] = Apprehend(π[state]*agents, R[state])
        Ĉ[state] = δ*R[state] + (λ-1)*agents*π[state] + agents*F*π[state]*A[state]
    return Ĉ  

### Table A.1

First cell produces the transition matrix of the finite regular Markov chain. Using the TM we produce the associated Markov chain (MC) and its associated stationary distribution ($\bar{S}$).

The following cell produces the results from Table A.1 by comparing the estimated distribution of violations ($\boldsymbol{\hat{D}}$) against $\bar{S}$.

In [5]:
""" 
    produce TM and SD in order to benchmark simulation
            state space for:
                    N = 50, Z=1
"""
# paramaters for benchmarking exercise
switch = True
agents, Z = 50, 1
R_low, R_high = 0, agents+1
# variables used for benchmark exercise
R = np.ones(np.sum(np.arange(1,agents+2)), dtype=int)

# "control" variables
ex_v = np.zeros(R_high-R_low)
ex_a = np.zeros(R_high-R_low)
std_v = np.zeros(R_high-R_low)
# vectors, arrays, storage objects
S̄ = np.empty(R_high-R_low, dtype=object)
V̂ = np.empty(R_high-R_low, dtype=object)
Â = np.empty(R_high-R_low, dtype=object)
Ĉ = np.empty(R_high-R_low, dtype=object)
MC = np.empty(R_high-R_low, dtype=object)

# create the ordered state space
S, Ŝ, s = State_space(agents)
max_v = np.zeros(len(s))
max_a = np.zeros(len(s))
for state in range(len(s)):
    max_v[state] = Ŝ[state][1]
    max_a[state] = Ŝ[state][2]

# create transition matrices
M1 , M2 = TM_matrices(agents, F)

# main loop
for r in range(R_low, R_high):
    index = 0
    a, v = 0, 0
    R.fill(r)
    TM = Transition_matrix(agents, M1, M2, R, F, switch)
    MC[r] = qe.MarkovChain(TM)
    S̄[r] = MC[r].stationary_distributions
    V̂[r] = S̄[r]*max_v
    Â[r] = S̄[r]*max_a
    ex_v[r] = np.sum(V̂[r])
    ex_a[r] = np.sum(Â[r])
    std_v[r] = np.std(V̂[r])

In [208]:
"""
        Benchmarking: Run simulation to converge for comparison with results from cell above
                        
                        Note: need to set r = 5, 21, 45 to reproduce Table A.1
"""
runs, block = 1000, 50000
agents = 50
Z = 1
r = 45
C, checks = 0.01, 5
TEST = np.zeros(runs)

# translate SD from TM into same format as DD1
#      DD1 runs from 0 to agents+1
#      state space of SD is much larger
DD3 = np.zeros(agents+1)
for i in Ŝ:
    DD3[i[1]] = DD3[i[1]] + S̄[r][0][i[0]-1]

time_to_converge = 0
for run in range(runs):
    v_μ = 0
    converged, passed = False, 0
    DD1 = np.zeros(agents+1)
    DD2 = np.zeros(agents+1)
    criminal_opps = Criminal_opps(ḡ,σ,agents,block)
    v_t, a_t, g_t = Simulation(criminal_opps,r,F,Z)
    v, a, g = 0, 0, 0
    v = np.append(v, v_t)
    a = np.append(a, a_t)
    g = np.append(g, g_t)
    for i in v:
        DD1[i] = DD1[i] + 1
    while not converged:
        time_to_converge = time_to_converge + 1
        criminal_opps = Criminal_opps(ḡ,σ,agents,block)
        v_t, a_t, g_t = Simulation(criminal_opps,r,F,Z)
        v = np.append(v, v_t)
        a = np.append(a, a_t)
        g = np.append(g, g_t)
        for i in v:
            DD2[i] = DD2[i] + 1    
        if np.abs(np.sum((DD2/np.sum(DD2)) - (DD1/np.sum(DD1)))) < C:
            passed = passed + 1
            if passed > checks:
                converged = True
        DD1 = DD2[:]
    DD1 = DD1/np.sum(DD1)
    TEST[run] = np.sum(np.abs(DD1-DD3))
    
x = np.mean(TEST)
y = np.std(TEST)
z = np.max(TEST)

np.save(path+'TEST.npy', TEST)

np.set_printoptions(precision=4)
print("%0.4f" % x),print("%0.4f" % y),print("%0.4f" % z),print("%0.4f" % ((time_to_converge+1)/1000))

0.0087
0.0011
0.0134
6.0010


(None, None, None, None)